# Final Deaths-Emissions 2015

In [1]:
import pandas as pd
import numpy  as np
import csv
from glob import glob

Leemos el fichero Excel de Complejos Industriales geolocalizados y filtramos por Emisiones al Aire

In [2]:
FEmissions = pd.ExcelFile('../data/csv/Emissions/geo/Comp_by_Emi_total_Geo_2001-2015.xlsx', sheetname='Comp_by_Emi_total_Geo_2001-2015')
DfEmissions = pd.read_excel(FEmissions,index_col=None,na_values=['NA'])
DfEmissions = DfEmissions[(DfEmissions['MedioReceptor'] == 'Aire')]

In [3]:
#Listado único de emisiones
ListEmissions = DfEmissions["Contaminante"].unique()
del DfEmissions

Iteración que recorre el listado de Contaminantes y se genera un fichero CSV para cada uno de ellos con registros agrupados
por defunción-emisión durante los años de exposición a cada Contaminante. Se filtra por el año de defunción 2015. 
El tiempo de ejecución es de 10 a 15' dependiendo de la máquina

El output es un fichero CSV por Contaminante agrupado por defunción (Id), Cantidad Kg exposición y total años exposición

In [4]:
for i in range(0,len(ListEmissions)): 
    NomFich = ListEmissions[i].replace('(', '')
    NomFich = NomFich.replace(')', '')
    NomFich = NomFich.replace('/', '-')
    NomFich = NomFich.replace(' ', '_')
    NomFich = NomFich.replace(',', '_')
    NomFich = NomFich.replace('á', 'a')
    NomFich = NomFich.replace('í', 'i')
    NomFich = NomFich.replace('ó', 'o')
    NomFich = NomFich.replace('Ó', 'O')
    NomFich = NomFich.replace('Contaminante_', 'C_')

    #Listado de ficheros entrada DeathsEmissions por emisión/
    ListFichDeathsEmissionsFinal = glob('../data/csv/group/DeathsEmissions_'+str(NomFich)+'*.csv')

    DfAcumuladoEmissions  = pd.DataFrame()
    cont = 0
    print ('Limpiando nombre fichero Contaminante Num: ',i)
    
    for i2 in range(0,len(ListFichDeathsEmissionsFinal)):   
        
         print ('Fichero DeathsEmission num: ',i2)
         #cargamos df con cada fichero  
         DfDeathsEmissions = pd.read_csv(ListFichDeathsEmissionsFinal[i2],sep=';',compression='gzip') 
         
         # Nos quedamos sólo con las defunciones ocurridas durante el año 2015, por cuestiones de optimización de memoria proyecto.
         DfDeathsEmissions = DfDeathsEmissions[DfDeathsEmissions["AnioDefuncion"] == 2015]
        
         # Añade columna sumatorio de emisiones por ID, elimina duplicados por id y año de referencia puesto
         # que pueden existir varias empresas para el mismo contaminante y misma defunción
         DfTotales = DfDeathsEmissions.groupby('Id').agg({'CantidadTotalkgporanyo':'sum'})
         DfTotales = DfTotales.rename(columns={"CantidadTotalkgporanyo":"CantidadTotalkg"})
         DfDeathsEmissions = DfDeathsEmissions.merge(DfTotales, how='left', left_on='Id', right_index=True)
         DfDeathsEmissions = DfDeathsEmissions.drop_duplicates(('Id','AnyoReferencia'), keep="first", inplace=False)

         # Añade columna total años exposición contaminante para cada defunción
         DfTotales = DfDeathsEmissions.groupby('Id').agg({'Id':'count'})
         DfTotales = DfTotales.rename(columns={"Id":"TotalAniosId"})
         DfDeathsEmissions = DfDeathsEmissions.merge(DfTotales, how='left', left_on='Id', right_index=True)
         DfDeathsEmissions = DfDeathsEmissions.drop_duplicates('Id', keep="first", inplace=False)

         DfAcumuladoEmissions = DfAcumuladoEmissions.append(DfDeathsEmissions)

         DfTotales = DfAcumuladoEmissions.groupby('Id').agg({'CantidadTotalkg':'sum'})
         DfAcumuladoEmissions.drop(['CantidadTotalkg'], axis=1, inplace=True)
         DfAcumuladoEmissions = DfAcumuladoEmissions.merge(DfTotales, how='left', left_on='Id', right_index=True)
         DfAcumuladoEmissions = DfAcumuladoEmissions.drop_duplicates(('Id'), keep="first", inplace=False)

         DfTotales = DfAcumuladoEmissions.groupby('Id').agg({'TotalAniosId':'sum'})
         DfAcumuladoEmissions.drop(['TotalAniosId'], axis=1, inplace=True)
         DfAcumuladoEmissions = DfAcumuladoEmissions.merge(DfTotales, how='left', left_on='Id', right_index=True)
         DfAcumuladoEmissions = DfAcumuladoEmissions.drop_duplicates('Id', keep="first", inplace=False)
 
         print ('Longitud dataframe Acumulado Emisiones: ',len(DfAcumuladoEmissions))
 
            
    if len(DfAcumuladoEmissions) > 0:   

        print(NomFich)  
        #elimina columnas no significativas y que han perdido su valor real por las agrupaciones
        DfAcumuladoEmissions.drop(['Unnamed: 0',
                                   'AnyoReferencia',
                                   'CantidadTotalkgporanyo'
                                    ], axis=1, inplace=True)
        
       
        DfAcumuladoEmissions.to_csv('../data/csv/final/DeathsEmissions_final_'+str(NomFich)+'_1.csv', \
                   sep=';', encoding='utf-8', compression='gzip')
        del DfDeathsEmissions    
        del DfTotales   
        del DfAcumuladoEmissions 
        

Limpiando nombre fichero Contaminante Num:  0
Fichero DeathsEmission num:  0
Longitud dataframe Acumulado Emisiones:  127148
Fichero DeathsEmission num:  1
Longitud dataframe Acumulado Emisiones:  151514
Fichero DeathsEmission num:  2
Longitud dataframe Acumulado Emisiones:  171029
Fichero DeathsEmission num:  3
Longitud dataframe Acumulado Emisiones:  180292
Fichero DeathsEmission num:  4
Longitud dataframe Acumulado Emisiones:  185633
Fichero DeathsEmission num:  5
Longitud dataframe Acumulado Emisiones:  185914
Fichero DeathsEmission num:  6
Longitud dataframe Acumulado Emisiones:  186273
Fichero DeathsEmission num:  7
Longitud dataframe Acumulado Emisiones:  190610
Fichero DeathsEmission num:  8
Longitud dataframe Acumulado Emisiones:  191800
Amoniaco_NH3
Limpiando nombre fichero Contaminante Num:  1
Fichero DeathsEmission num:  0
Longitud dataframe Acumulado Emisiones:  126462
Fichero DeathsEmission num:  1
Longitud dataframe Acumulado Emisiones:  138657
Fichero DeathsEmission num

Longitud dataframe Acumulado Emisiones:  5615
Triclorobencenos_totales_TCB
Limpiando nombre fichero Contaminante Num:  44
Fichero DeathsEmission num:  0
Longitud dataframe Acumulado Emisiones:  3265
1_2_3_4_5_6-hexaclorociclohexano_HCH
Limpiando nombre fichero Contaminante Num:  45
Fichero DeathsEmission num:  0
Longitud dataframe Acumulado Emisiones:  5615
1_1_1-tricloroetano_TCE
